# Networking - TCP, IP and DNS

---

## 1. DNS, IP, and TCP basics

The aim of this section is to build a clear mental model of how one program finds and talks to another over a network. Everything from REST APIs to trading gateways sits on top of the same core ideas: names are turned into IP addresses, IP addresses and ports identify programs, and TCP sockets carry bytes reliably between them.

### DNS: turning names into IP addresses

Humans prefer names such as `google.com`, but routers and switches only understand numeric IP addresses like `142.250.x.y`. DNS, the Domain Name System, is the distributed phonebook that converts these human‑readable names into machine‑usable IP addresses. When your code wants to contact `google.com`, it first asks a DNS resolver, “What is the IP for this domain?”. The resolver replies with one or more IP addresses, and from that point onward your operating system communicates directly with the IP, not the name. Conceptually, every network conversation begins with the same pattern: domain name goes in, DNS returns an IP, and the IP is then used to open a connection.

### IP and ports: which machine, which program

An IP address tells the network which machine should receive a packet, but a single machine runs many programs at once. To distinguish between them, the networking stack uses ports, which are just integer labels attached to sockets. An IP answers the question “which host?” while a port answers “which program on that host?”. For example, `127.0.0.1` refers to your own machine, often called localhost. If an echo server is listening on port `5001`, then the pair `127.0.0.1:5001` uniquely identifies that particular server process. When a client connects to `127.0.0.1:5001`, the IP ensures the traffic stays on the local machine, and the port tells the kernel which process should receive the bytes. A useful analogy is a building and apartments: the building address is the IP, and the apartment number is the port.

### TCP sockets: a reliable byte stream

Once the client knows the IP and port of the server, it still needs a protocol to move data back and forth in a safe and predictable way. TCP, the Transmission Control Protocol, provides a reliable, ordered stream of bytes between two endpoints. It is connection‑oriented: before any data flows, client and server perform a three‑way handshake (SYN, SYN‑ACK, ACK) to agree that a connection exists and to set up state on both sides. It guarantees that bytes are delivered in the same order in which they were sent, automatically retransmitting lost packets and reordering out‑of‑sequence packets so the application sees a clean stream or a failed connection, but never corrupted ordering.

TCP sockets are full‑duplex, meaning both ends can send and receive simultaneously, much like a two‑way pipe. In the echo server example, the server creates a TCP socket, binds it to `127.0.0.1:5001`, and calls `listen()`. The operating system now knows that any incoming TCP connection on that IP and port should be handed to this process. When the client creates its own TCP socket and calls `connect(("127.0.0.1", 5001))`, the OS completes the three‑way handshake with the server. Once `connect` returns, both sides treat the connection as established. The client calls `sendall(b"hello networking")`, TCP breaks that into packets and delivers them reliably, and the server’s `recv(1024)` call returns exactly that byte string. When the server sends the same bytes back, the client’s `recv` sees its own message echoed.

From the Python code’s perspective, this feels almost trivial: write bytes, read bytes. All of the complexity of packet boundaries, retransmissions, and reordering is handled by TCP and the operating system. This simple abstraction—a reliable byte stream between two endpoints—is the foundation on which higher‑level protocols such as HTTP, WebSockets, FIX, and many proprietary trading feeds are built. Those protocols are essentially structured messages layered on top of this TCP stream.


In [1]:
!tcpdump -i lo tcp port 5001

/bin/bash: line 1: tcpdump: command not found


In [3]:
import socket
import threading

def echo_server(host="127.0.0.1", port=5001):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind((host, port))
        s.listen()
        print(f"[server] listening on {host}:{port}")
        conn, addr = s.accept()
        with conn:
            print("[server] connected by", addr)
            while True:
                data = conn.recv(1024)
                if not data or data == b"quit":
                    print("[server] shutting down")
                    break
                print("[server] received:", data)
                conn.sendall(data)

thread = threading.Thread(target=echo_server, daemon=True)
thread.start()
print("[main] server started in background")


[server] listening on 127.0.0.1:5001
[main] server started in background


In [5]:
import socket

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect(("127.0.0.1", 5001))
    s.sendall(b"hello")
    print("[client] got:", s.recv(1024))
    s.sendall(b"quit")  # tells server to exit its loop


[server] connected by ('127.0.0.1', 54980)
[server] received: b'hello'
[client] got: b'hello'
[server] shutting down
